In [1]:
# Import packages for later use
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV


# Modules for Random Forests:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from scipy.special import boxcox1p

# Additional imported modules:
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn import tree

import graphviz

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load data
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')


numerical_features = train_data.dtypes[train_data.dtypes != 'object'].index.values
categorical_features = train_data.dtypes[train_data.dtypes == 'object'].index.values

# Additional values to omit based on results of featurer importance from Random Forests:
# 'BsmtFinSF2', 'ScreenPorch','LowQualFinSF',  'BsmtHalfBath',  'MiscVal', 'PoolArea',  '3SsnPorch'
omit = ['SalePrice', 'Id', 'Training']
submit = ['SalePrice', 'Id']

In [3]:
# Preprocess

# drop one row in training
train_data.dropna(subset=['Electrical'], inplace=True)

# concat
train_data['Training'] = 1
test_data['Training'] = 0
all_data = pd.concat([train_data, test_data], ignore_index=True)

# fill categoricals
categoricals = all_data[categorical_features]
categoricals.fillna(0, inplace=True)
all_data[categorical_features] = categoricals

# fill numericals with mean:
numericals = all_data[numerical_features]
numericals.fillna(numericals.mean(), inplace=True)
all_data[numerical_features] = numericals

# fill numericals with 0
# all_data.fillna(0, inplace=True)

# process categoricals
all_data = pd.get_dummies(data=all_data)


In [4]:
# dropping a few columns
to_drop = ['MoSold', 'PoolArea', 'GarageYrBlt', '3SsnPorch', 'YearRemodAdd', 'LowQualFinSF']
all_data.drop(columns=to_drop, inplace=True)

train_set = all_data.loc[all_data['Training'] == 1]
test_set = all_data.loc[all_data['Training'] == 0]
# drop saleprice
all_data.drop(columns='SalePrice', inplace=True)

In [5]:
# obtain X & y
y = train_set['SalePrice']
y_log = np.log(y)

X = train_set[[c for c in train_set.columns if c not in omit]]
X_test = test_set[[c for c in test_set.columns if c not in omit]]

In [6]:
train_set

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,196.0,706.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,0.0,978.0,0.0,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,162.0,486.0,0.0,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,0.0,216.0,0.0,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,350.0,655.0,0.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1456,60,62.0,7917,6,5,1999,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,1457,20,85.0,13175,6,6,1978,119.0,790.0,163.0,...,0,0,0,1,0,0,0,0,1,0
1456,1458,70,66.0,9042,7,9,1941,0.0,275.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1457,1459,20,68.0,9717,5,6,1950,0.0,49.0,1029.0,...,0,0,0,1,0,0,0,0,1,0


In [7]:
rt = DecisionTreeRegressor(criterion='mse', max_depth=5)

model_r = rt.fit(X, y_log)

In [8]:
# Output of Regression Tree:

y_log_pred = np.exp(rt.predict(X_test))

test_data['SalePrice'] = y_log_pred
submission = test_data[[c for c in test_data.columns if c in submit]]
submission.to_csv('./data/teamJarvis_rt_2.csv', index=False)

In [9]:
fig = plt.figure(figsize=(200,200))
tree.plot_tree(rt)

[Text(5580.0, 9966.0, 'X[3] <= 6.5\nmse = 0.16\nsamples = 1459\nvalue = 12.024'),
 Text(2790.0, 8154.0, 'X[13] <= 1378.5\nmse = 0.082\nsamples = 911\nvalue = 11.814'),
 Text(1395.0, 6342.0, 'X[10] <= 799.5\nmse = 0.07\nsamples = 564\nvalue = 11.704'),
 Text(697.5, 4530.0, 'X[223] <= 0.5\nmse = 0.085\nsamples = 184\nvalue = 11.51'),
 Text(348.75, 2718.0, 'X[31] <= 0.5\nmse = 0.11\nsamples = 45\nvalue = 11.245'),
 Text(174.375, 906.0, 'mse = 0.075\nsamples = 41\nvalue = 11.307'),
 Text(523.125, 906.0, 'mse = 0.036\nsamples = 4\nvalue = 10.615'),
 Text(1046.25, 2718.0, 'X[13] <= 1134.5\nmse = 0.047\nsamples = 139\nvalue = 11.596'),
 Text(871.875, 906.0, 'mse = 0.043\nsamples = 77\nvalue = 11.503'),
 Text(1220.625, 906.0, 'mse = 0.028\nsamples = 62\nvalue = 11.711'),
 Text(2092.5, 4530.0, 'X[5] <= 1947.5\nmse = 0.035\nsamples = 380\nvalue = 11.798'),
 Text(1743.75, 2718.0, 'X[4] <= 3.5\nmse = 0.037\nsamples = 49\nvalue = 11.573'),
 Text(1569.375, 906.0, 'mse = 0.004\nsamples = 3\nvalue = 1

In [10]:
# Random Forests: 
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y_log, test_size=0.3)

n_estimators = [int(x) for x in np.arange(start=10, stop=1000, step=10)]
max_features = [0.5, 'auto', 'sqrt', 'log2']
min_samples_split = st.randint(1, 11)
min_samples_leaf = st.randint(1, 11)
bootstrap = [True, False]

# Param List to feed for Randomized Search CV, cv (cross-validation splitting strategy) variable not provided in dict,
# by default if cv not provided RandomizedSearchCV uses 5-Fold Cross Validation:
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

# First create the base model to tune
rf = RandomForestRegressor()
# Fit the random search model
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=params, n_iter=100, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train_split, y_train_split)
print(rf_random.best_params_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'bootstrap': True, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 810}


In [11]:
# Random Forests, params based on best_params output from Randomized Search Model:
rf = RandomForestRegressor(n_estimators=920, random_state=42, n_jobs=-1, oob_score=True, min_samples_leaf=2,
                           max_features=0.5, min_samples_split=2, bootstrap=True)

rf.fit(X, y_log)

RandomForestRegressor(max_features=0.5, min_samples_leaf=2, n_estimators=920,
                      n_jobs=-1, oob_score=True, random_state=42)

In [12]:
y_log_pred = np.exp(rf.predict(X_test))

test_data['SalePrice'] = y_log_pred
submission = test_data[[c for c in test_data.columns if c in submit]]
submission.to_csv('./data/teamJarvis_random_forests.csv', index=False)

In [13]:
# Seeing the importance of features:

importance = pd.DataFrame(rf.feature_importances_,index=X.columns,columns=["Importance"])
importance.sort_values(by=['Importance'],ascending=False)

,Importance
OverallQual,0.331060
GrLivArea,0.163507
YearBuilt,0.084558
GarageCars,0.046005
TotalBsmtSF,0.042344
...,...
Utilities_AllPub,0.000000
RoofMatl_Roll,0.000000
Condition2_RRAn,0.000000
Exterior1st_0,0.000000


In [14]:
# Discarding features with less importance:
not_important = importance[importance['Importance'] == 0].index

print(not_important)


X = X[[c for c in X.columns if c not in not_important]]
X_test = X_test[[c for c in X_test.columns if c not in not_important]]

Index(['MSZoning_0', 'Utilities_0', 'Utilities_AllPub', 'Utilities_NoSeWa',
       'LotConfig_FR3', 'Condition1_RRNe', 'Condition1_RRNn',
       'Condition2_Artery', 'Condition2_Feedr', 'Condition2_PosA',
       'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'RoofStyle_Shed', 'RoofMatl_ClyTile', 'RoofMatl_Membran',
       'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_0',
       'Exterior1st_AsphShn', 'Exterior1st_CBlock', 'Exterior1st_ImStucc',
       'Exterior1st_Stone', 'Exterior2nd_0', 'Exterior2nd_CBlock',
       'Exterior2nd_ImStucc', 'Exterior2nd_Other', 'ExterCond_Ex',
       'ExterCond_Po', 'Foundation_Wood', 'Heating_Floor', 'HeatingQC_Po',
       'Electrical_Mix', 'KitchenQual_0', 'Functional_0', 'Functional_Sev',
       'PoolQC_Ex', 'PoolQC_Fa', 'PoolQC_Gd', 'MiscFeature_Gar2',
       'MiscFeature_Othr', 'MiscFeature_TenC', 'SaleType_0', 'SaleType_CWD',
       'SaleType_Con', 'SaleType_Oth'],
      dtype='object')


In [15]:
# Random Forests: 
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y_log, test_size=0.3)

n_estimators = [int(x) for x in np.arange(start=10, stop=1000, step=10)]
max_features = [0.5, 'auto', 'sqrt', 'log2']
min_samples_split = st.randint(1, 11)
min_samples_leaf = st.randint(1, 11)
bootstrap = [True, False]

# Param List to feed for Randomized Search CV, cv (cross-validation splitting strategy) variable not provided in dict,
# by default if cv not provided RandomizedSearchCV uses 5-Fold Cross Validation:
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap}

# First create the base model to tune
rf = RandomForestRegressor()
# Fit the random search model
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=params, n_iter=100, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train_split, y_train_split)
print(rf_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'bootstrap': False, 'max_features': 0.5, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 920}


In [20]:
#Random Forests Output - Discarding less important features:
#Random Forests, params based on best_params output from Randomized Search Model:
rf = RandomForestRegressor(n_estimators=920, random_state=42, n_jobs=-1, oob_score=True, min_samples_leaf=2,
                           max_features=0.5, min_samples_split=2, bootstrap=True)

rf.fit(X, y_log)
y_log_pred = np.exp(rf.predict(X_test))

test_data['SalePrice'] = y_log_pred
submission = test_data[[c for c in test_data.columns if c in submit]]
submission.to_csv('./data/teamJarvis_random_forests_imp.csv', index=False)

In [17]:
# Gradient Boosting Regressor:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y_log, test_size=0.3)

n_estimators = [int(x) for x in np.arange(start=10, stop=1000, step=10)]
max_features = [0.5, 'auto', 'sqrt', 'log2']
min_samples_split = st.randint(1, 11)
min_samples_leaf = st.randint(1, 11)
bootstrap = [True, False]

# Param List to feed for Randomized Search CV, cv (cross-validation splitting strategy) variable not provided in dict,
# by default if cv not provided RandomizedSearchCV uses 5-Fold Cross Validation:
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split,
        }

# First create the base model to tune
gb = GradientBoostingRegressor()
# Fit the random search model
gb_random = RandomizedSearchCV(estimator=gb, param_distributions=params, n_iter=100, verbose=2, random_state=42, n_jobs=-1)
gb_random.fit(X_train_split, y_train_split)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 920}


In [25]:
# Parameters based on previous runs of CV Search:
gb = GradientBoostingRegressor(n_estimators=630, random_state=42, min_samples_leaf=1, max_features='sqrt', min_samples_split=4)

gb.fit(X, y_log)

GradientBoostingRegressor(max_features='sqrt', min_samples_split=4,
                          n_estimators=630, random_state=42)

In [26]:
y_log_pred = np.exp(gb.predict(X_test))

test_data['SalePrice'] = y_log_pred
submission = test_data[[c for c in test_data.columns if c in submit]]
submission.to_csv('./data/teamJarvis_gb2.csv', index=False)